Mit diesem Jupyter Notbook werden bibl. Daten der Exemplarebene von der K10plus-HAB-Schnittstelle http://sru.k10plus.de/opac-de-23 abgerufen und Datenfelder der SRU-Antwort mit Hilfe von XPath-Abfragen gezielt durchsucht sowie deren Inhalt an eine CSV-Datei übergeben. Das Notebook ist im Rahmen der Masterarbeit "Digitale Erschließung, Analyse und Visualisierung der
Handbibliothek von Herzogin Philippine Charlotte von Braunschweig-Lüneburg. Forschungsdatenmanagement in Kooperation
zwischen Forschung und Bibliothek." im Studiengang Digitales Datenmanagement der FH Potsdam/HU Berlin entstanden.

Die nachfolgenden Funktionen sind zum weiteren Verständnis auskommentiert.
Ich danke Dario Kampkasper (DK) für seine Expertise und Unterstützung beim Erstellen des Codes.
2023-02-20 Henrike Fricke-Steyer

In [ ]:
#Import der benötigten Python-Bibliothken
import pandas as pd
import requests 
from lxml import etree
import csv

In [142]:
#Einlesen einer Datei mit Identifikationsnummern (IDNs) und Umwandlung der Spalte IDN in Liste die auf dem eigenen 
#Computer liegt, der Pfad "C:/...csv" muss entsprechend ersetzt werden: 
colnames=['IDN']

ns = {
    "pica": "info:srw/schema/5/picaXML-v1.0"
}

data = pd.read_csv("C:/...csv", names=colnames, header=None)
idns = data['IDN'].to_list()
print(len(idns))

1326


In [143]:
#gekürzte Funktion zur Abfrage der IDNs aus dem SRU-Tutorial (www.dnb.de/dnblabtutorials)
#Schleife zur SRU-Abfrage, spricht die Schnittstelle an, Parameter in entsprechender Reihenfolge angepasst
def sru(query):
    
    base_url = "http://sru.k10plus.de/opac-de-23"
    params = {'version': '1.1',
              'operation': 'searchRetrieve',                    
              'query': query,
              'maximumRecords': '100',
              'recordSchema' : 'picaxml'
         }
    r = requests.get(base_url, params=params)
    
    ns = {
        "zs":"http://www.loc.gov/zing/srw/",
        "pica": "info:srw/schema/5/picaXML-v1.0"
    }
    xml = etree.fromstring(r.content)
    records = xml.xpath("//zs:recordData/pica:record", namespaces=ns)
    
    return records
    

In [144]:
#Übergabe der einzelnen Elemente aus der IDN-Liste an die Funktion, um für jedes Element eine SRU-Abfrage 
#auszulösen. Die Ergebnisse der einzelnen Abfragen werden als Liste in der Variable "response" gespeichert
response = [sru(searchtext) for searchtext in idns]

#Ausgabe der Anzahl der in der Liste enthaltenen Elemente - diese sollte mit der Anzahl der abgefragten
#IDNs übereinstimmen. 
print (len(response))

1326


In [145]:
#Ausgabe der gesammelten Treffer in einer Datei
# 2022-10-21 DK
resultTree = etree.ElementTree()
recordsElement = etree.Element('records')
resultTree._setroot(recordsElement)

for record in response: 
    recordsElement.append(record[0])

with open ('data.xml', 'wb',) as f:
    resultTree.write(f, encoding='utf-8')

In [146]:
#Hilfsfunktion, um ein wiederholbares Fedl abzurufen (inkl. Unterfelder)
# 2023-01-21 DK
def get_repeatable ( xml, *fields ):
    values = []
    for field in fields:
        xpath = "pica:datafield[@tag='" + field + "']"
        values.extend(xml.xpath(xpath, namespaces=ns))
    
    return values

In [147]:
# Hilfsfunktion, um für eine Liste von Unterfeldern die Werte abzurufen
#2023-01-21 DK 
def get_subfields ( xml, subfields ):
    values = []
    for subfield in subfields:
        xpath = "pica:subfield[@code = '" + subfield + "']/text()"
        try:
            val = xml.xpath(xpath, namespaces=ns)
            values.append(val[0])
        except:
            "leer"
    
    if len(values):
        return values[0]
    else:
        return "leer"

In [148]:
# Hilfsfunktion, um Unterfelder gezielt abzurufen
# 2023-01-21 DK
# Es wird immer nur das erste gefundene zurückgegeben, daher nur für nicht wiederholbare Felder verwenden
def get_field ( xml, field, subfields ):
    values = []
    for subfield in subfields:
        xpath = "pica:datafield[@tag='" + field + "']/pica:subfield[@code = '" + subfield + "']/text()"
        try:
            value = xml.xpath(xpath, namespaces=ns)[0]
            values.append(value)
        except:
            "leer"
    
    try:
        return next(value for value in values if value)
    except:
        "leer"

In [149]:
#Extraktion entsprechender Inhalte aus den einzelnen Records: 
# Funktion 2022-10-21 DK

def parse_record(record):
    xml = record[0]
 
    
    # ppn / 100
    idn = xml.xpath("pica:datafield[@tag='003@']/pica:subfield[@code = '0']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    

# Signaturen 209A=7100 kann wiederholt werden
    weitereSgnKategorien = get_repeatable(xml, '209A') 
    weitereSgn = []
    for kat in weitereSgnKategorien:
        weitereSgn.append([
            get_subfields(kat, ['a', 'A']),
           
        ])
    
    sgn1 = "leer"
    sgn2 = "leer"
    sgn3 = "leer"
    sgn4 = "leer"
    sgn5 = "leer"
     

    try:
        sgn1 = weitereSgn[0][0]
        sgn2 = weitereSgn[1][0]
        sgn3 = weitereSgn[2][0]
        sgn4 = weitereSgn[3][0]
        sgn5 = weitereSgn[4][0]
      
        
    except:
        pass
    
      
    
# Anmerkungen 237A=4801 kann wiederholt werden
    weitereAnmKategorien = get_repeatable(xml, '237A') 
    weitereAnm = []
    for kat in weitereAnmKategorien:
        weitereAnm.append([
            get_subfields(kat, ['a', 'A']),
           
        ])
    
    anm1 = "leer"
    anm2 = "leer"
    anm3 = "leer"
     

    try:
        anm1 = weitereAnm[0][0]
        anm2 = weitereAnm[1][0]
        anm3 = weitereAnm[2][0]
        
    except:
        pass
    
# Provenienzen 244Z=6800 kann wiederholt werden
    weiterePrvKategorien = get_repeatable(xml, '244Z') 
    weiterePrv = []
    for kat in weiterePrvKategorien:
        weiterePrv.append([
            get_subfields(kat, ['a', 'A']),
           
        ])
    
    prv1 = "leer"
    prv2 = "leer"
    prv3 = "leer"
     

    try:
        prv1 = weiterePrv[0][0]
        prv2 = weiterePrv[1][0]
        prv3 = weiterePrv[2][0]
        
    except:
        pass
        
    meta_dict = {
        "PPN":idn,
        "Signatur 1":sgn1,
        "Signatur 2":sgn2,
        "Signatur 3":sgn3,
        "Signatur 4":sgn4,
        "Signatur 5":sgn5,
        "Anmerkung 1":anm1,
        "Anmerkung 2":anm2,
        "Anmerkung 3":anm3,
        "Provenienz 1":prv1,
        "Provenienz 2":prv2,
        "Provenienz 3":prv3
    }
    
    return meta_dict

In [150]:
#Übergabe der einzelnen Records an die Funktion und Ausgabe als DataFrame:
output = [parse_record(record) for record in response]

# print(output[0])
df = pd.DataFrame(output)

In [151]:
#Ausgabe des DataFrames in CSV-Datei
df.to_csv('sru_antwort.csv', sep ='\t') 